# Max Flow Applications

The purpose of this assignment is to investigate applications of finding a Max Flow. The problem asks you to design and implement an algorithm for shipping a material between nodes with different supply and demand requirements.

## Movie distribution

First solve Problem 3 from HW3-theoretical. 

Now suppose a movie distributor would like to ship a copy of a film from CA to every other state. There are therefore 48 units to ship out of CA, and each other state receives 1 unit. 

The dataset contiguous-usa.dat lists the adjacent states in the US. Each line lists two adjacent states; thus AK and HI are omitted, but DC is included in the data. The following code reads in the graph of US states.

In [ ]:
import networkx as nx
G = nx.Graph()

usa = open('contiguous-usa.dat')#open file 
for line in usa:
    s1, s2 = line.strip().split()
    G.add_edge(s1, s2)

We now encode the demands into the graph.

In [ ]:
for state in G.nodes():
    if state != 'CA':
        G.node[state]['demand'] = 1
G.node['CA']['demand'] = -48

We will assign a uniform capacity of 16 to each edge. Since CA has only three adjacent states, this is the smallest possible uniform capacity that allows one to ship all 48 units out of CA. As we have created an undirected graph, and flows have directions, we first convert the graph to a directed graph.

In [ ]:
G = nx.DiGraph(G)
uniform_capacity = 16
for (s1, s2) in G.edges():
    G.edge[s1][s2]['capacity'] = uniform_capacity

Complete the following function to implement your algorithm to find a flow with demands. Your function should work correctly for any input, not just the movie instance considered here. As always, you are encouraged to define auxiliary functions as needed for clarity.

In [ ]:
def flow_with_demands(graph):
    for state in graph.nodes(): #traverse all nodes in graph
        if  graph.node[state]['demand'] > 0:
            graph.add_edge(state, 'T')# if node's demand>0, create a sink 'T' and link this node with T
            graph.edge[state]['T']['capacity'] = graph.node[state]['demand']
            #every new edge has capacity that equals demand
    for state in graph.nodes():
        if state!='T':
            if  graph.node[state]['demand'] < 0:
                graph.add_edge('S', state)# if node's demand<0, create a source 'S' and link this node with S
                graph.edge['S'][state]['capacity'] = -graph.node[state]['demand']
                #every new edge has capacity that equals -demand
    flow_value, flow_dict = nx.maximum_flow(graph, 'S', 'T')#make this new graph run max flow function
    for state in graph.nodes():
        if state!='T'and state!='S':
            if graph.node[state]['demand'] < 0:  
                graph.remove_edge('S',state)#remove all edge that connected with S
    graph.remove_node('S')#remove node S in graph
    for state in graph.nodes():
        if state!='T':
            if graph.node[state]['demand'] > 0:
                graph.remove_edge(state,'T')#remove all edge that connected with T in graph
    graph.remove_node('T')  #remove node T in graph
    del flow_dict['T']
    del flow_dict['S']    # delete key S and T in flow_dict       
    for key1 in flow_dict:
        for key2 in flow_dict[key1]:
            if key2=='T'or key2=='S':
                break
        if key2=='T' or key2=='S':
            del flow_dict[key1][key2]      #delete all edge that connected with S or T in flow_dict
    a=0
    for state in graph.nodes():
        if  graph.node[state]['demand'] < 0:
            a=a+graph.node[state]['demand']      #calculate the sum of supply nodes'demands 
    b=0
    for state in graph.nodes():
        if  graph.node[state]['demand'] > 0:
            b=b+graph.node[state]['demand']      #calculate the sum of sink nodes'demands 
    if a+b==0 and flow_value==-a:   
        return flow_dict
        # if the sum of supply nodes' demands equals the sum of sink nodes'demands and max flow equals one of them
        # then return flow_dict
    else:
        raise nx.NetworkXUnfeasible('No flow satisfying all demands.')
        #otherwise, raise 'No flow satisfying all demands.'

To verify that your solution is correct, implement a function that computes the total flow into each node (which will be negative for supply nodes).

In [ ]:
def divergence(flow):
    dict1={}                #create a new dict
    for key in flow:
        flow_out=0
        flow_in=0
        for key1 in flow[key]:
            flow_out=flow_out+flow[key][key1] #calculate the total out flow in every node
        for key2 in flow:
            for key3 in flow[key2]:
                if key3==key:
                    flow_in=flow_in+flow[key2][key3] #calculate the total in flow in every node
        dict1[key]=flow_in-flow_out   
    return dict1

The following code performs a sanity check on your function (but does not completely confirm correctness).

In [ ]:
flow = flow_with_demands(G)
div = divergence(flow)
print "Flow satisfies all demands:", all(div[n] == G.node[n]['demand'] for n in G.nodes())